In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import bond_pricing 
import bond_pricing as bp

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bond_pricing/simple_bonds.py:529: UserWarning: Module isda_daycounters is not installed.
Only 'simple' daycount (basically ACT/365) is available.
To use other daycounts, install isda_daycounters from
https://github.com/miradulo/isda_daycounters
  warn("Module isda_daycounters is not installed.\n"
